In [1]:
import numpy as np
from pandas import read_csv, DataFrame

NumPy buat operasi matriks<br>
read_csv & DataFrame dari Pandas buat muat <i>dataset</i> & bikin DataFrame baru

# Muat <i>dataset</i>

In [2]:
data = read_csv("seeds_data.csv")
data.head()

,area,perimeter,compactness,length of kernel,width of kernel,asymmetry coefficient,length of kernel groove,class
0,15.2,14.8,0.870,5.76,3.31,2.22,5.20,1
1,14.8,14.5,0.881,5.55,3.33,1.01,4.95,1
2,14.2,14.0,0.900,5.29,3.33,2.69,4.82,1
3,13.8,13.9,0.895,5.32,3.37,2.25,4.80,1
4,16.1,14.9,0.903,5.65,3.56,1.35,5.17,1


Datanya diambil dari sini : https://archive.ics.uci.edu/ml/datasets/seeds<br>
Sudah dibersihkan secara manual untuk mengisi nilai yang kosong dengan metode <i>simple rolling mean</i>

In [3]:
cls = data['class']
data = data.drop(axis=1, columns='class')

data.head()

,area,perimeter,compactness,length of kernel,width of kernel,asymmetry coefficient,length of kernel groove
0,15.2,14.8,0.870,5.76,3.31,2.22,5.20
1,14.8,14.5,0.881,5.55,3.33,1.01,4.95
2,14.2,14.0,0.900,5.29,3.33,2.69,4.82
3,13.8,13.9,0.895,5.32,3.37,2.25,4.80
4,16.1,14.9,0.903,5.65,3.56,1.35,5.17


Oke karena kita ini mau nerapin <i>unsupervised learning</i> jadi'nya kolom <b>class</b> di <i>dataset</i> kita itu nggak dipake ya

# Inisialisasi nilai centroid acak

In [4]:
centroid = np.zeros([len(np.unique(cls)), data.shape[1]])

for i in range(len(centroid)):
    centroid[i] = data.sample()
    
centroid

array([[18.9  , 16.6  ,  0.85 ,  6.54 ,  3.6  ,  3.69 ,  6.49 ],
       [18.8  , 16.2  ,  0.896,  6.08 ,  3.76 ,  1.64 ,  6.1  ],
       [11.4  , 13.2  ,  0.826,  5.3  ,  2.69 ,  5.38 ,  5.3  ]])

Nah jadi jumlah centroid itu ndak ada ketentuan'nya ya asal ndak melebih'i jumlah data aja.<br>
Karena di <i>dataset</i> ini jumlah kategori/kelas'nya ada tiga yaudah saya bikin tiga centroid<br>dengan jumlah data percentroid sama dengan jumlah kolom pada <i>dataset</i>.<br>
Banyak'nya centroid sama dengan banyaknya klaster yang terbentuk.<br>
    
Oke mungkin ada yang bertanya "Apa tidak ada metode buat nentuin jumlah centroid yang ngepas sama <i>dataset</i> kita?"<br>
Ada! Pake aja <b>Elbow Method</b>.<br>
Selengkapnya : https://en.wikipedia.org/wiki/Elbow_method_(clustering)<br>


# K-Means Clustering

<img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/4efcba672e6df32cc8eb7ce0863591806a6581b5">
Kita pake <i>euclidean distance</i> aja ya<br>
walaupun ndak menutup kemungkinan bisa juga pake rumus <i>distance</i> yang lain sih hehehe...<br>

Jadi p itu nilai yang ada di data (kolom ke <sub>n</sub>)<br>
Terus q itu nilai centroid (kolom ke <sub>n</sub>)

In [5]:
# PROGRAM UTAMA

while True:
    # BUAT NAMPUNG HASIL PERHITUNGAN SEMENTARA
    hasil = DataFrame(0, dtype = 'float',
                      index = np.arange(len(data)),
                      columns = ['C1', 'C2', 'C3'])

    # EUCLIDEAN DISTANCE
    for i in range(len(data)):
        for j in range(len(centroid)):
            hasil.iloc[i][j] = np.sqrt(np.sum((data.loc[i]-centroid[j])**2))
    
    # BUAT NAMPUNG NILAI CENTROID BARU
    new_centroid = np.zeros([len(np.unique(cls)),
                             data.shape[1]])
    
    # KOLOM BARU BUAT NAMPUNG NILAI KLASTER
    data['cluster'] = np.argmin(hasil.values,
                                axis=1)

    # CARI NILAI RATA-RATA DATA DARI KLASTER YANG TERBENTUK
    for i in range(len(centroid)):
        new_centroid[i] = np.average(data[data['cluster'] == i],
                                     axis=0)[:-1]
    
    # KALAU NILAI CENTROID BARU SAMA DENGAN CENTROID LAMA SAMA MAKA LOOP BERHENTI
    if np.all(centroid == new_centroid):
        del hasil
        break
    # KALAU NILAINYA NGGAK SAMA
    else:
        # KOLOM cluster PADA data TERHAPUS &
        data = data.drop(axis=1,columns=['cluster'])
        # NILAI CENTROID BARU DIPAKAI BUAT PERHITUNGAN SELANJUTNYA
        centroid = new_centroid

Oke dari program di atas kita sudah nerapin K-Means Clustering, berikut adalah hasilnya

In [6]:
new_centroid

array([[18.66393443, 16.22295082,  0.88267213,  6.19983607,  3.71442623,
         3.59655738,  6.05393443],
       [14.52054795, 14.36027397,  0.86553425,  5.55506849,  3.26780822,
         2.62958904,  5.18287671],
       [11.85394737, 12.87368421,  0.85081579,  5.21434211,  2.86210526,
         4.76197368,  5.08197368]])

# Data yang sudah memiliki klaster

In [7]:
data

,area,perimeter,compactness,length of kernel,width of kernel,asymmetry coefficient,length of kernel groove,cluster
0,15.2,14.8,0.870,5.76,3.31,2.22,5.20,1
1,14.8,14.5,0.881,5.55,3.33,1.01,4.95,1
2,14.2,14.0,0.900,5.29,3.33,2.69,4.82,1
3,13.8,13.9,0.895,5.32,3.37,2.25,4.80,1
4,16.1,14.9,0.903,5.65,3.56,1.35,5.17,1
...,...,...,...,...,...,...,...,...
205,12.1,13.0,0.878,5.13,2.98,3.63,4.80,2
206,11.2,12.8,0.851,5.10,2.79,4.32,5.00,2
207,13.0,13.6,0.888,5.23,3.23,8.31,5.05,2
208,11.8,13.2,0.852,5.17,2.83,3.59,5.04,2


# Klaster yang terbentuk

In [8]:
data['cluster'].ravel()

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 2, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 1, 1,
       1, 1, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 1, 1, 0, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2])

Untuk <i>unsupervised learning itu</i> klaster yang terbentuk dapat kita namai sendiri.<br>
Kalau diambil dari <i>output</i> di atas bisa saja angka 0 dganti menjadi 3.<br>
Itu dilakukan agar urutan'nya sama dengan kolom <b>class</b> pada <i>dataset</i> yang asli.

In [9]:
cls.ravel()

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])